<a href="https://colab.research.google.com/github/Lukehsu1999/Academic-and-Formality-Rewriter/blob/main/SubstitutionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries needed

In [ ]:
import string
import re

<h2>Replacement Dictionaries</h2>

In [ ]:
#Basic Taboo Dicts
sentence_start_dict={"DictName":"sentence_start", 'Also':"Moreover,", 'So':"Therefore", 'And':"#AND#", 'Besides':"Additionally,"} #And: should be removed
vague_word_dict={"DictName":"vague_word", 
      'stuff':"#sTUFF", 'Stuff':"#STUFF", 'thing':"#tHING", 'Thing':"#THING"}
      #'anything':"#ANYTHING#", 'Anything':"#ANYTHING#", 'nothing':"#NOTHING#",'Nothing':"#NOTHING#", 
      #'something':"#SOMETHING#", 'Something':"#Something", 'everything':"#everything", 'Everything':"#Everything"}

#she's: she is OR she has
#he'd: he had OR he would
#we'd: we had OR we would OR we should
#we're: we are OR we were
#it's: it is OR it has
#who's: who is OR who has
informal_contraction_dict={"DictName":"contraction", 
              "I'd":"#I'D", 
              "I'll":"I will","i'll":"I will", "I've":"I have","i've":"I have","I'm":"I am","i'm":"I am", 
              "you're":"you are", "You're":"You are", "you'll":"you will", "You'll":"You will",
              "you've":"you have", "You've":"You have",
              "you'd":"#yOU'D","You'd":"#YOU'D",
              "she's":"#sHE'S", "She's":"#SHE'S", "he's":"#hE'S", "He's":"#HE'S",
              "she'd":"#sHE'D", "he'd":"#hE'D",
              "She'd":"#SHE'D", "He'd":"#HE'D",  
              "she'll":"she will", "he'll":"he will",
              "She'll":"She will", "He'll":"He will", 
              "they'll":"they will", "they're":"they are", "they've":"they have", 
              "They'll":"They will", "They're":"They are", "They've":"They have",
              "they'd":"#tHEY'D", "They'd":"#THEY'D", 
              "we'd":"#wE'D","We'd":"#WE'D",
              "we're":"we are",
              "We're":"We are",
              "we'll":"we will", "we've":"we have",
              "We'll":"We will", "We've":"We have",
              "it's":"#iT'S", "It's":"#IT'S",
              "it'd":"#iT'D","It'd":"#IT'D",
              "it'll":"it will", "that's":"that is", "there's":"there is", "where's":"where is", "let's":"let us",
              "It'll":"It will", "That's":"That is", "There's":"There is", "Where's":"Where is", "Let's":"Let us",
              "that'll":"that will", "That'll":"That will",
              "that'd":"that would", "That'd":"That would",
              "can't":"can not", "couldn't":"could not", "didn't":"did not", "doesn't":"does not", "don't":"do not",
              "Can't":"Can not", "Couldn't":"Could not", "Didn't":"Did not", "Doesn't":"Does not", "Don't":"Do not",
              "hadn't":"had not", "hasn't":"has not", "isn't":"is not", "shouldn't":"should not", "wasn't":"was not",
              "Hadn't":"Had not", "Hasn't":"Has not", "Isn't":"Is not", "Shouldn't":"Should not", "Wasn't":"Was not", 
              "haven't":"have not", "Haven't":"Have not",
              "aren't":"are not", "Aren't":"Are not",
              "weren't":"were not", "Weren't":"Were not",
              "wouldn't":"would not", "Wouldn't":"Would not",
              "won't":"will not", "wouldn't":"would not", "could've":"could have", "might've":"might have", "must've":"must have",
              "Won't":"Will not", "Wouldn't":"Would not", "Could've":"Could have", "Might've":"Might have", "Must've":"Must have",
              "should've":"should have", "would've":"would have",
              "Should've":"Should have", "Would've":"Would have",
              "who's":"#wHO'S", "Who's":"#WHO'S",
              "who'll":"who will", "Who'll":"Who will",
              "who'd":"#wHO'D", "Who'd":"#WHO'D",
              "why'd":"why would", "Why'd":"Why would",
              "how's":"how is", "How's":"How is",
              "how'd":"how would","How'd":"How would",
              "what's":"what is", "What's":"What is",
              "here's":"here is", "Here's":"Here is"}

informal_others_dict={"DictName":"others", "till":"untill", "Till":"Untill"}
informal_frequency_dict={"DictName":"frequency", 
              "always":"#aLWAYS", "Always":"#ALWAYS", "never":"#nEVER", "Never":"#NEVER"}
#too: I am a student, too. vs. he is too awesome
informal_intensifier_dict={"DictName":"intensifier", "very":"", "Very":"", "extremely":"", "Extremely":"", "really":"#rEALLY", "Really":"#REALLY", 
              "too":"#tOO", "Too":"#TOO", "so":"#sO"}

In [ ]:
#incomplete taboo dicts: will not substitute them for now (Feb 23)
informal_subjective_dict={"DictName":"subjective", "naturally":"", "Naturally":"#NATURALLY", "obviously":"clearly", "Obviously":"Clearly"}
good_bad_dict={"DictName":"good_bad", "good":"good", "Good":"#GOOD", "bad":"#bAD", "Bad":"#BAD"}
superlatives_dict={"DictName":"superlatives", "best":"#bEST", "Best":"#BEST", "worst":"#wORST", "Worst":"#WORST"}
personal_preference_dict={"DictName":"personal_preference", "like":"#lIKE", "likes":"#lIKES", "liked":"#lIKED", "love":"#lOVE", "loves":"#lOVES", "loved":"#lOVED"}

In [ ]:
#extended taboo dicts: will not substitute them for now (Feb 23)
informal_second_person_dict={"DictName":"second_person", "you":"you", "You":"You", "your":"your", "Your":"Your"}
first_person_dict={"DictName":"first_person", "I":"I", "We":"We", "we":"we"}

For informal phrases dict

In [ ]:
def match_with_phrase_dict(words, i, word):
  if i+1<len(words) and (word=='a' or word=='A') and words[i+1]=='bit': 
    return "a bit"
  elif i+2<len(words) and (word=="a" or word=='A') and words[i+1]=="lot" and words[i+2]=="of":
    return 'a lot of'
  elif i+2<len(words) and (word=="a" or word=='A') and words[i+1]=="couple" and words[i+2]=="of":
    return 'a couple of'
  elif i+1<len(words) and (word=="kind" or word=="Kind") and words[i+1]=="of":
    return 'kind of'
  elif i+1<len(words) and word=="sort" and words[i+1]=="of":
    return 'sort of'
  elif i+1<len(words) and (word=="of" or word=="Of") and words[i+1]=="course":
    return 'of course'
  elif i+2<len(words) and (word=="a" or word=="A") and words[i+1]=="long" and words[i+1]=="time":
    return 'a long time'
  elif i+1<len(words) and (word=="a" or word=="A") and words[i+1]=="while":
    return 'a while'
  else:
    return -1

Things to think about substituing sentences: </br>
word sequence need to be the same <br>
puctuation need to be preserved <br>
there might be multiple words to substitute in the sentence -> index will change <br>
There might be duplicate target words in the sentence: you should've won, you really should've become the winner <br>

<h2>Substitution Function </h2>

In [ ]:
def get_substituted_sentence(sentence_set, replace_dict_set):
  new_line_set=[]
  for line in sentence_set:
    #preprocessing
    mypunct=string.punctuation[0:6]+string.punctuation[7:len(string.punctuation)] #except '
    punctless_sentence=line.translate(str.maketrans('','',mypunct))
    words=punctless_sentence.split()
    new_line=line
    for i,word in enumerate(words):
      for replace_dict in replace_dict_set:
        if word in replace_dict.keys():
          #print(word)
          #print(replace_dict[word])
          new_line=new_line.replace(word,replace_dict[word])
    new_line_set.append(new_line)

  return new_line_set


<h2>Execution</h2>

Replacement Dictionaries

In [ ]:
basic_replace_dict_set=[sentence_start_dict,
        informal_contraction_dict, informal_frequency_dict, 
        informal_intensifier_dict, informal_others_dict]
#remove vague_word_dict temporarily
incomplete_taboo_dict_set=[informal_subjective_dict, good_bad_dict,
        superlatives_dict, personal_preference_dict]
extended_taboo_dict_set=[informal_second_person_dict, first_person_dict]

replace_dict_set=[]
replace_dict_set+=basic_replace_dict_set
#replace_dict_set+=incomplete_taboo_dict_set
#replace_dict_set+=extended_taboo_dict_set

Get corpus

In [ ]:
#path to the dataset
path_to_dataset="/content/drive/MyDrive/CityU/FYP/Mar29Split/test/FR/formal"

#open file
f=open(path_to_dataset,"r")
lines=f.readlines()
partial_lines=lines[0:10]
print(partial_lines)

['I mean that you have to really be her friend.\n', 'Are you posing a rhetorical question?\n', 'Men pretend to love in order to have intercourse; women pretend to enjoy intercourse in order to get love.\n', 'I do not intend to be mean.\n', 'I would estimate an average of 45% initially but then, once you become acquainted with the person, perhaps 15%.\n', 'Because some women send subtle messages to men rather than being clear.\n', 'Let us purchase coffee and converse and proceed from that point!\n', 'Also, i dislike it when my father is unhappy.\n', 'Ask him if you should go see a doctor.\n', 'You can post more questioins on Yahoo! answers.\n']


Get replace sentence

In [ ]:
result=get_substituted_sentence(lines,replace_dict_set)

Save to file

In [ ]:
path_to_write_file="/content/drive/MyDrive/CityU/FYP/Mar29Split/test/FR/formal_auto_substituted"
f=open(path_to_write_file,"w") #w: overwrite; a: append
for lines in result:
  f.write(lines)
f=open(path_to_write_file,"r")
lines=f.readlines()
print(lines[0:10])

['I mean that you have to #rEALLY be her friend.\n', 'Are you posing a rhetorical question?\n', 'Men pretend to love in order to have intercourse; women pretend to enjoy intercourse in order to get love.\n', 'I do not intend to be mean.\n', 'I would estimate an average of 45% initially but then, once you become acquainted with the person, perhaps 15%.\n', 'Because some women send subtle messages to men rather than being clear.\n', 'Let us purchase coffee and converse and proceed from that point!\n', 'Moreover,, i dislike it when my father is unhappy.\n', 'Ask him if you should go see a doctor.\n', 'You can post more questioins on Yahoo! answers.\n']


<h2> Exception: Phrase Dict </h2>
<p> will manually update it </p>

In [ ]:
phrase_dict={"DictName":"phrases", 'a bit':'somewhat', 'a lot of':'a great number of', 
      'a couple of':'several', 'kind of':'somewhat', 'sort of':'somewhat', "of course":"clearly", 
      'a long time':"#a LONG TIME#", 'a while':"#a WHILE#"}

In [ ]:
path_to_write_file="/content/drive/MyDrive/CityU/FYP/Datasets/GYAFC/GYAFC_Corpus/Entertainment_Music/train/grammarlylized"
f=open(path_to_write_file,"r")
lines=f.readlines()
print(lines[-10:-1])

UnicodeDecodeError: ignored